# Badminton Serve Fault Detector

In [28]:
import mediapipe as mp 
import os 
import uuid #name our image
import cv2 
import numpy as np 
from matplotlib import pyplot as plt 
import pygame

# Initialize Pygame mixer for audio
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound("fart-01.wav")
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [9]:
def draw_angles(a,b,c): 
    #loop through diff hands 
    
             
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0]) 
    angle=np.abs(radians*180.0/np.pi) 
    if angle>180: 
        angle=360-180 
   
    return angle     

In [29]:
def draw_angles(a, b, c):
    # Calculate angles in left quadrants
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.degrees(radians)
    
    if angle < 0:
        angle += 360
    
    return angle


In [ ]:
def check_hand_position(l):
    hip_y = pose_landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
    wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    return wrist_y > waist_y

In [36]:
# VIDEO FEED 
cap = cv2.VideoCapture(0)  
# curl counter 
counter = 0 
stage = None  # play or fault
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): 
        ret, frame = cap.read() 
        # Detect stuff and render 
        # Convert BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip horizontally
        image = cv2.flip(image, 1)

        # Set writeable flag to False
        image.flags.writeable = False

        # Process detections
        results = pose.process(image)

        # Set writeable flag to True
        image.flags.writeable = True

        # Convert RGB back to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        # Extract landmarks 
        try: 
            landmarks = results.pose_landmarks.landmark 
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]  
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]  
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]  
            lthumb = [landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x, landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
             
            # Calculate angle 
            elbow_angle = draw_angles(shoulder, elbow, wrist) 
            wrist_angle = draw_angles(elbow, wrist, lthumb)
            if elbow_angle >= 0 and elbow_angle <= 180: 
                stage = "READY"
            if elbow_angle > 180: 
                stage = "PLAY" 
                if wrist_angle>227: 
                    stage="FAULT" 
                    alarm_sound.play()
            if elbow_angle > 235 and stage == "PLAY": 
                stage = "FAULT!" 
                alarm_sound.play()
                
            # Visualize 
            cv2.putText(image, str(round(elbow_angle, 2)), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (16, 255, 16), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(wrist_angle, 2)), tuple(np.multiply(wrist, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (16, 255, 16), 2, cv2.LINE_AA)
         
        except: 
            pass 
        # Setup status box 
        cv2.rectangle(image, (0, 0), (326, 73), (16, 117, 255), -1)  # (img,start,end,colour,-1 is box is coloured)   
        # Stage data 
        cv2.putText(image, "STAGE", (105, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA) 
        cv2.putText(image, stage, (100, 60), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA) 
        
        # Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), 
                                  mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)) 
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('e'): 
            break 
cap.release()
cv2.destroyAllWindows()
